In [ ]:
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

In [ ]:
def open_text(fn:PathOrStr):
    with open(fn,'r') as f: return ''.join(f.readlines())

In [ ]:
def _treat_html(o:str)->str:
    return o.replace('\n','\\n')

def _text2html_table(items:Collection[Collection[str]], widths:Collection[int])->str:
    html_code = f"<table>"
    for w in widths: html_code += f"  <col width='{w}%'>"
    for line in items:
        html_code += "  <tr>\n"
        html_code += "\n".join([f"    <th>{_treat_html(o)}</th>" for o in line if len(o) >= 1])
        html_code += "\n  </tr>\n"
    return html_code + "</table>\n"

In [ ]:
class Text(ItemBase):
    def __init__(self, ids, text): self.data,self.text = ids,text
    def __str__(self):  return str(self.text)
    
    def show_batch(self, idxs:Collection[int], rows:int, ds:Dataset, figsize:Tuple[int,int]=(9,10))->None:
        from IPython.display import clear_output, display, HTML
        items = [['text', 'label']]
        for i in idxs[:rows]:
            x,y = ds[i]
            items.append([str(x), str(y)])
        display(HTML(_text2html_table(items, [90,10])))

class NumericalizedTextList(ItemList):
    def __init__(self, items:Iterator, vocab:Vocab=None, create_func:Callable=None, path:PathOrStr='.', xtra=None):
        super().__init__(items, create_func=create_func, path=path, xtra=xtra)
        self.vocab = vocab
        
    def new(self, items:Iterator, xtra:Any=None)->'NumericalizedTextList':
        return self.__class__(items=items, vocab=self.vocab, create_func=self.create_func, path=self.path, xtra=xtra)
    
    def get(self, o): return Text(o, self.vocab.textify(o))
    
class TokenizedTextList(NumericalizedTextList):
    
    def preprocess(self, vocab:Vocab=None, max_vocab:int=60000, min_freq:int=2):
        self.vocab = ifnone(vocab, Vocab.create(self.items, max_vocab, min_freq))
        self.preprocess_kwargs = {'vocab': self.vocab}
        self.items = np.array([self.vocab.numericalize(t) for t in self.items])
    
class TextList(TokenizedTextList):
    def preprocess(self, tokenizer:Tokenizer=None, chunksize:int=10000, vocab:Vocab=None, 
                 max_vocab:int=60000, min_freq:int=2):
        tokenizer = ifnone(tokenizer, Tokenizer())
        tokens = []
        for i in progress_bar(range(0,len(self.items),chunksize), leave=False):
            tokens += tokenizer.process_all(self.items[i:i+chunksize])
        self.items = tokens
        super().preprocess(vocab, max_vocab, min_freq)
    
class TextFilesList(TextList):
    def __init__(self, items:Iterator, create_func:Callable=None, path:PathOrStr='.'):
        texts = [open_text(fn) for fn in items]
        super().__init__(texts, create_func, path)

In [ ]:
il = (TextList.from_csv(path, 'texts.csv', create_func=None, col='text')
      .random_split_by_pct()
      .label_from_df(CategoryList, cols=0))

In [ ]:
sd = il.preprocess()

In [ ]:
len(sd.valid.vocab.itos), len(sd.train.vocab.itos)

(6176, 6176)

In [ ]:
data = sd.databunch()

In [ ]:
data.show_batch()

text,label
i watched lion king more times that all my friends put xxunk . having a baby sister .. you know how it is . by now i xxunk both the plot and the lines . after lion king 2 came out i was like ok well let me see ... the second one was significantly weaker ... then i saw an ad for lion king 1 and 1 / 2 ... i was like ok there we go again . after watching the 1 1 / 2 i was like wow . all my expectations ( for xxunk ) were broken . a truly lovely and original plot keeps you glued to your seat for the entire time . i have noticed that the cartoon was filled with so many comical moments that xxunk will apply here 100 % . \n\n i xxunk recommend seeing the cartoon .,positive
"oh , this is so bad , it is funny . the only way one could explain something like this is a porn party with drugs that resulted in the resolution to make a movie just for fun . i mean : you get to see porn actresses topless , having sex , then killed by human xxunk . there is plenty of gore , including the classic "" something is wrong with her , oh no , it 's half the person she used to be "" and the xxunk murder caused by panic . but you can also find funny stuff like intestines pulled through someone 's ass and a guy running in the woods then finding himself decapitated by a wire tied between two trees ( that makes a xxunk xxunk sound afterward , like in cartoons ) . somehow there is a market for people going beyond porn , they really need to know what 's inside an actress , mere xxunk are not enough . therefore you get to see plenty of summary xxunk on xxunk xxunk . there is no dialog . jenna and xxunk have really small parts and i really wonder what richard xxunk wanted when xxunk a role here . i think this is a film one must watch with the button on fast forward and watch only the xxunk stuff , just to be reminded of the old school c class horror movies .",negative
"well , what are the odds ! at the exact right moment that a few xxunk amateur - scientists discover xxunk paintings xxunk that some type of dinosaur monster might have inhabited the area thousands of years ago , a burning xxunk xxunk into the lake and xxunk xxunk a monster 's xxunk that has been lying there for over a thousand years , i suppose ! "" the crater lake monster "" is a movie that literally must be seen to be believed , but you better do so in the company of many friends and a pile of xxunk in order to make the xxunk a little bit easier to xxunk . yes , this is a terrible film with the xxunk xxunk screenplay xxunk and numerous irrelevant padding xxunk that are downright embarrassing , but it 's also xxunk charming and so clumsily put together that you simply have to cherish some kind of xxunk for it . half of the film  at least  revolves on the wacky adventures of xxunk and mitch . these two local xxunk own and run a boat renting shop near the lake , but spend most of their days picking their xxunk and xxunk over fascinating stuff like to spell the word "" bait "" . it is mostly during their xxunk laurel & hardy situations that new xxunk of blood or decapitated heads are discovered in the lake . steve xxunk , the heroic but not exactly sharp sheriff is on the case , but only if he 's not too busy chasing big city thugs traveling through the area . halfway through the film , there suddenly is an abrupt scene about a thug xxunk a xxunk store and killing two people in the process . this xxunk "" xxup xxunk "" moment appears to take on the complete other side of the country , like in new york city or something , and has absolutely nothing to do with the events going on at crater lake . only like twenty minutes later the xxunk pops up again in xxunk and there 's an "" xxunk "" chase through the woods , ending in the xxunk 's hungry xxunk . the absurd little details in "" the crater lake monster "" are too numerous to mention ! for example , this is probably the only creature - feature in which the players discover the obligatory xxunk xxunk xxup after they already xxunk the act